In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymongo import ASCENDING, GEOSPHERE, MongoClient
import pandas as pd
from alive_progress import alive_bar
from shapely.geometry import Point, mapping
from keplergl import KeplerGl
import shapely
import json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import geopandas as gpd
from h3 import h3
import math
import sklearn
import numpy as np
import time

In [16]:
RESOLUTION = 11

In [5]:
client = MongoClient('mongodb://localhost:27017/')
db = client.osmDataDB
coll_cities = db.cities
coll_relations = db.relations
coll_cache = db.hexEmbeddingCache

In [6]:
city = coll_cities.find_one({'city': 'Wrocław'})

In [12]:
shp = shapely.geometry.shape(city['geometry'])
buffered_polygon = shp.buffer(0.005)

In [17]:
indexes = h3.polyfill(shapely.geometry.mapping(buffered_polygon), RESOLUTION, geo_json_conformant=False)
len(indexes)

289883

In [20]:
# hexes = []
hexes_shp = []
for h in tqdm(indexes):
    polygons = h3.h3_set_to_multi_polygon([h], geo_json=False)
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polygon = shapely.geometry.Polygon(polyline)
    if shp.intersects(polygon):
        cache_count = coll_cache.find({ 'hex_id': h }).count()
        # hexes.append({'hex_id': h, 'geometry': shapely.geometry.mapping(polygon)})
        hexes_shp.append({'hex_id': h, 'geometry': polygon, 'count': cache_count })

  0%|          | 0/289883 [00:00<?, ?it/s]<ipython-input-20-72ed47bdec27>:9: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cache_count = coll_cache.find({ 'hex_id': h }).count()
100%|██████████| 289883/289883 [14:13<00:00, 339.49it/s]


In [21]:
df = gpd.GeoDataFrame(hexes_shp)
del df['hex_id']
df.to_file(f"wrocław_hexes_cache_{RESOLUTION}.geojson", driver='GeoJSON')